# Exercise 02. Create a Custom Multi-Agent RLlib Environment

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this tutorial, you will learn how to:
 * [Code a custom RLlib Multi-agent environment](#multi_agent_env)
 * [Select an algorithm and instantiate a config object using that algorithm's config class](#rllib_algo)
 * [Train a RL model using a multi-agent capable algorithm from RLlib](#rllib_run)



In [20]:
# import required packages

import time
import gym
from gym.spaces import Discrete, MultiDiscrete
import numpy as np
from ipywidgets import Output
from IPython import display

import ray
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray import tune
from ray.tune.logger import pretty_print

print(f"gym: {gym.__version__}")
print(f"ray: {ray.__version__}")

# !ale-import-roms --import-from-pkg atari_py.atari_roms

gym: 0.21.0
ray: 3.0.0.dev0


## Code a custom RLlib Multi-agent environment <a class="anchor" id="multi_agent_env"></a>

We will create the following (adversarial) multi-agent environment.  Wil will use this custom environment for the rest of this tutorial.

<img src="images/multi_agent_arena_0.png" width="80%">
<img src="images/multi_agent_arena_1.png" width="80%">
<img src="images/multi_agent_arena_2.png" width="80%">
<img src="images/multi_agent_arena_3.png" width="80%">



### Review OpenAI Gym Environments

We learned in the last lesson about OpenAI Gym Environments.  Specifically we covered Gym:
<ul>
    <li>Action Space</li>
    <li>Observation Space</li>
    <li>Rewards</li>
    <li><i>`done`</i> signal</li>
    <li>Gym Environment API methods:</li>
    <ul>
        <li>reset(self)</li>
        <li>step(self, action: dict)</li>
        <li>render(self, mode=None)</li>
    </ul>
    </ul>

### RLlib MultiAgentEnv API

RLlib supports environments created using the OpenAI Gym API. This means, to create a RLlib environment from scratch, you need to minimally implement the same methods [required by Gym](https://www.gymlibrary.ml/content/api/#standard-methods).  

We want a multi-agent environment, so we will implement the [RLlib MultiAgentEnv base class](https://github.com/ray-project/ray/blob/master/rllib/env/multi_agent_env.py) which requires a few extra methods in addition to the minimal Gym methods.
    
<ul>
    <li>__init__(self)</li>
    <li> _get_obs(self)</li>
    <li>_move(self, coords, action, which_agent)</li>
    </ul>

In [21]:
# Let's code our multi-agent environment

class MultiAgentArena(MultiAgentEnv):  # MultiAgentEnv is a gym.Env sub-class
    
    def __init__(self, config=None):
        config = config or {}
        # Dimensions of the grid.
        self.width = config.get("width", 10)
        self.height = config.get("height", 10)

        # End an episode after this many timesteps.
        self.timestep_limit = config.get("ts", 100)

        self.observation_space = MultiDiscrete([self.width * self.height,
                                                self.width * self.height])
        # 0=up, 1=right, 2=down, 3=left.
        self.action_space = Discrete(4)

        # Reset env.
        self.reset()
        
        # For rendering.
        self.out = None
        if config.get("render"):
            self.out = Output()
            display.display(self.out)

        self._spaces_in_preferred_format = False

    def reset(self):
        """Returns initial observation of next(!) episode."""
        # Row-major coords.
        self.agent1_pos = [0, 0]  # upper left corner
        self.agent2_pos = [self.height - 1, self.width - 1]  # lower bottom corner

        # Accumulated rewards in this episode.
        self.agent1_R = 0.0
        self.agent2_R = 0.0

        # Reset agent1's visited fields.
        self.agent1_visited_fields = set([tuple(self.agent1_pos)])

        # How many timesteps have we done in this episode.
        self.timesteps = 0

        # Did we have a collision in recent step?
        self.collision = False
        # How many collisions in total have we had in this episode?
        self.num_collisions = 0

        # Return the initial observation in the new episode.
        return self._get_obs()

    def step(self, action: dict):
        """
        Returns (next observation, rewards, dones, infos) after having taken the given actions.
        
        e.g.
        `action={"agent1": action_for_agent1, "agent2": action_for_agent2}`
        """
        
        # increase our time steps counter by 1.
        self.timesteps += 1
        # An episode is "done" when we reach the time step limit.
        is_done = self.timesteps >= self.timestep_limit

        # Agent2 always moves first.
        # events = [collision|agent1_new_field]
        events = self._move(self.agent2_pos, action["agent2"], is_agent1=False)
        events |= self._move(self.agent1_pos, action["agent1"], is_agent1=True)

        # Useful for rendering.
        self.collision = "collision" in events
        if self.collision is True:
            self.num_collisions += 1
            
        # Get observations (based on new agent positions).
        obs = self._get_obs()

        # Determine rewards based on the collected events:
        r1 = -1.0 if "collision" in events else 1.0 if "agent1_new_field" in events else -0.5
        r2 = 1.0 if "collision" in events else -0.1

        self.agent1_R += r1
        self.agent2_R += r2
        
        rewards = {
            "agent1": r1,
            "agent2": r2,
        }

        # Generate a `done` dict (per-agent and total).
        dones = {
            "agent1": is_done,
            "agent2": is_done,
            # special `__all__` key indicates that the episode is done for all agents.
            "__all__": is_done,
        }

        return obs, rewards, dones, {}  # <- info dict (not needed here).

    def _get_obs(self):
        """
        Returns obs dict (agent name to discrete-pos tuple) using each
        agent's current x/y-positions.
        """
        ag1_discrete_pos = self.agent1_pos[0] * self.width + \
            (self.agent1_pos[1] % self.width)
        ag2_discrete_pos = self.agent2_pos[0] * self.width + \
            (self.agent2_pos[1] % self.width)
        return {
            "agent1": np.array([ag1_discrete_pos, ag2_discrete_pos]),
            "agent2": np.array([ag2_discrete_pos, ag1_discrete_pos]),
        }

    def _move(self, coords, action, is_agent1):
        """
        Moves an agent (agent1 iff is_agent1=True, else agent2) from `coords` (x/y) using the
        given action (0=up, 1=right, etc..) and returns a resulting events dict:
        Agent1: "new" when entering a new field. "bumped" when having been bumped into by agent2.
        Agent2: "bumped" when bumping into agent1 (agent1 then gets -1.0).
        """
        orig_coords = coords[:]
        # Change the row: 0=up (-1), 2=down (+1)
        coords[0] += -1 if action == 0 else 1 if action == 2 else 0
        # Change the column: 1=right (+1), 3=left (-1)
        coords[1] += 1 if action == 1 else -1 if action == 3 else 0

        # Solve collisions.
        # Make sure, we don't end up on the other agent's position.
        # If yes, don't move (we are blocked).
        if (is_agent1 and coords == self.agent2_pos) or (not is_agent1 and coords == self.agent1_pos):
            coords[0], coords[1] = orig_coords
            # Agent2 blocked agent1 (agent1 tried to run into agent2)
            # OR Agent2 bumped into agent1 (agent2 tried to run into agent1)
            return {"collision"}

        # No agent blocking -> check walls.
        if coords[0] < 0:
            coords[0] = 0
        elif coords[0] >= self.height:
            coords[0] = self.height - 1
        if coords[1] < 0:
            coords[1] = 0
        elif coords[1] >= self.width:
            coords[1] = self.width - 1

        # If agent1 -> "new" if new tile covered.
        if is_agent1 and not tuple(coords) in self.agent1_visited_fields:
            self.agent1_visited_fields.add(tuple(coords))
            return {"agent1_new_field"}
        # No new tile for agent1.
        return set()

    def render(self, mode=None):

        if self.out is not None:
            self.out.clear_output(wait=True)

        print("_" * (self.width + 2))
        for r in range(self.height):
            print("|", end="")
            for c in range(self.width):
                field = r * self.width + c % self.width
                if self.agent1_pos == [r, c]:
                    print("1", end="")
                elif self.agent2_pos == [r, c]:
                    print("2", end="")
                elif (r, c) in self.agent1_visited_fields:
                    print(".", end="")
                else:
                    print(" ", end="")
            print("|")
        print("‾" * (self.width + 2))
        print(f"{'!!Collision!!' if self.collision else ''}")
        print("R1={: .1f}".format(self.agent1_R))
        print("R2={: .1f} ({} collisions)".format(self.agent2_R, self.num_collisions))
        print()
        time.sleep(0.25)


<br>
In the cell below:
<ul>
    <li>Initialize the environment</li>
    <li>Make both agents take a few steps</li>
    <li>Render the environment after each agent takes a step.</li>
    </ul>


In [22]:
env = MultiAgentArena(config={"render": True})
obs = env.reset()

with env.out:
    # Agent1 moves down, Agent2 moves up.
    obs, rewards, dones, infos = env.step(action={"agent1": 2, "agent2": 0})
    env.render()

    # Agent1 moves right, Agent2 moves left.
    obs, rewards, dones, infos = env.step(action={"agent1": 1, "agent2": 3})
    env.render()

    # Agent1 moves right, Agent2 moves left.
    obs, rewards, dones, infos = env.step(action={"agent1": 1, "agent2": 3})
    env.render()

    # Agent1 moves down, Agent2 moves up.
    obs, rewards, dones, infos = env.step(action={"agent1": 2, "agent2": 0})
    env.render()


print("Agent1's x/y position={}".format(env.agent1_pos))
print("Agent2's x/y position={}".format(env.agent2_pos))
print("Env timesteps={}".format(env.timesteps))

Output()

Agent1's x/y position=[2, 2]
Agent2's x/y position=[7, 7]
Env timesteps=4


## Select an algorithm and instantiate a config object using that algorithm's config class <a class="anchor" id="rllib_algo"></a>

<ol>
    <li><a href="https://docs.ray.io/en/master/rllib/rllib-algorithms.html">Open RLlib docs</a></li>
    <li>Scroll down and click url of algo you're searching for, e.g. <i><b>PPO</b></i></li>
    <li>On the <a href=""https://docs.ray.io/en/master/rllib/rllib-algorithms.html#ppo>algo docs page </a>, click on the link <i><b>Implementation</b></i>.  This will open the <a href="https://github.com/ray-project/ray/blob/master/rllib/algorithms/ppo/ppo.py">algo code file on github</a>.</li>
    <li>Search the github code file for the word <i><b>config</b></i></li>
    <li>Typically the docstring example will show: </li>
    <ol>
        <li>Example code implementing RLlib API, then </li>
        <li>Example code implementing Ray Tune API.</li>
    </ol>
    <li>Scroll down to the config <b>__init()__</b> method</li>
    <ol>
            <li>Algorithm default hyperparameter values are here.</li>
    </ol>
    </ol>

In [23]:
# Config is an object instead of a dictionary since Ray version >= 1.13
from ray.rllib.algorithms.ppo import PPOConfig

# uncomment below to see the long list of specifically PPO default config values
# print(pretty_print(PPOConfig().to_dict()))

config = PPOConfig()
config.environment(env=MultiAgentArena)
config.rollouts(num_rollout_workers=4, num_envs_per_worker=1)
config.training(lr=0.00005, train_batch_size=4000)  # default values for this algorithm
config.multi_agent(
    policies=["policy1", "policy2"],
    policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "policy1" if agent_id == "agent1" else "policy2",
)
# Set the log level to DEBUG, INFO, WARN, or ERROR 
config.debugging(log_level="ERROR")


## Train a RL model using a multi-agent algorithm from RLlib <a class="anchor" id="rllib_run"></a>

In [25]:
# To start fresh, restart Ray in case it is already running
#if ray.is_initialized():
#    ray.shutdown()

ppo = config.build()

for _ in range(10):
    result = ppo.train()
    print(f"Sum of rewards for both agents R={result['episode_reward_mean']}")
    print(f"Agent1 R={result['policy_reward_mean']['policy1']}")
    print(f"Agent2 R={result['policy_reward_mean']['policy2']}")
    print()

print("Training completed.")


2022-07-24 15:19:56,429	INFO trainable.py:160 -- Trainable.setup took 12.896 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-07-24 15:19:56,431	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Sum of rewards for both agents R=-12.855000000000008
Agent1 R=-4.0375
Agent2 R=-8.817499999999983

Sum of rewards for both agents R=-6.81
Agent1 R=1.58125
Agent2 R=-8.391249999999985

Sum of rewards for both agents R=-3.050999999999993
Agent1 R=5.42
Agent2 R=-8.470999999999984

Sum of rewards for both agents R=0.28800000000001186
Agent1 R=8.605
Agent2 R=-8.316999999999984

Sum of rewards for both agents R=0.45600000000001295
Agent1 R=8.355
Agent2 R=-7.898999999999986

Sum of rewards for both agents R=1.7160000000000153
Agent1 R=8.79
Agent2 R=-7.073999999999987

Sum of rewards for both agents R=2.4420000000000126
Agent1 R=9.395
Agent2 R=-6.952999999999987

Sum of rewards for both agents R=1.9860000000000084
Agent1 R=8.785
Agent2 R=-6.798999999999986

Sum of rewards for both agents R=3.141000000000005
Agent1 R=9.665
Agent2 R=-6.523999999999989

Sum of rewards for both agents R=3.9270000000000045
Agent1 R=9.67
Agent2 R=-5.742999999999991

Training completed.


<div class="alert alert-block alert-success">
<b>Note</b> that in an adversarial multi-agent setup, an agent benefits from the other agent's failures and vice-versa: agents get harmed more (receive negative rewards) the better the other agent is doing.
    <br/>
This highlights some important aspects of multi-agent training:
    <br/>
<ul>
<li>From each agent's perspective, the environment is not as static as in respective single-agent scenarios (the other agent's behavior is probably harder to predict than the environment's own inherent dynamics/physics).</li>
<li>As one agent learns how to behave more intelligently, the other agent has to counter this new behavior of its opponent and become smarter as well, asoasf.</li>
    </ul>
    </div>


In [8]:
# To stop the Algorithm and release its blocked resources, use:
ppo.stop()


### Homework

 <img src="images/exercise_env_loop.png" width=500>
 
We already learned how to use an environment's `reset()` and `step()` calls to walk through a single agent environment: Call `reset()` once, continue using the returned observations to compute actions, pass these actions into consecutive calls to `step()` and stop when `step()` returns the `done=True` flag.

Let's do the same thing now in the multi-agent setting, using our `MultiAgentArena` class.
Remember that everything, from observations, over rewards, dones, and actions now become dictionaries mapping agent IDs to the individual agent's observation, reward, action, and done information.

Follow these instructions here to get this done:

1. `reset` the already created (variable `env`) environment to get the first (initial) observations for "agent1" and "agent2".
1. Enter an infinite while loop, in which you ..
1. .. compute the actions for "agent1" and "agent2" (using random sampling).
1. .. put the results of the action computations into an action dict (`{"agent1": [action1], "agent2": [action2]}`).
1. .. pass this action dict into the env's `step()` method.
1. .. check the returned `dones` dict for True (yes, episode is terminated) and if True, break out of the loop. Note here that you may also check the `dones` dict for the special "__all__" key and if `dones['__all__'] is True` -> the episode has ended for all agents.

**Good luck! :)**


Write your solution code into this following python cell here:

In [ ]:
import time
from ipywidgets import Output
from IPython import display
import time


# Leave the following as-is. It'll help us with rendering the env in this very cell's output.,
out = Output()
display.display(out)

with out:

    # Start coding here inside this `with`-block:
    # 1) Reset the env ...

    # 2) Enter an infinite while loop (in order to step through one episode) ...

        # 3) Calculate both agents' actions individually, using random sampling with the 
        # action space: e.g. `a1 = env.action_space.sample()`.

        # 4) Compile the actions dict from both individual agents' actions ...

        # 5) Send the actions dict to the env's `step()` method to receive: obs, rewards, dones, info dicts ...

        # 6) We'll do this together: Render the env.
        # Don't write any code here (skip directly to 7).
        out.clear_output(wait=True),
        time.sleep(0.08),
        env.render(),

        # 7) Check, whether the episde is done (take a look at the
        # `dones` dict returned from `step()`)
        # If dones["__all__"], break out of the while loop we entered in step 2).


# 8) Run it! :)


## References

 * [Hands-on RL with Ray’s RLlib](https://github.com/sven1977/rllib_tutorials/tree/main/ray_summit_2021)